In [1]:
## Importing Libraries
import pandas as pd
import numpy as np
import os

import datetime
from prettytable import PrettyTable
import pymysql
import yaml
from sqlalchemy import create_engine, text

In [2]:
## Root Directory to read the configuration files
root_dir = os.getcwd()

In [3]:
## Fucntions

# Function to read YAML file
def read_yaml(file_path):
    with open(file_path, 'r') as file:
        try:
            # Load the YAML content into a Python dictionary
            data = yaml.safe_load(file)
            return data
        except yaml.YAMLError as exc:
            print(f"Error reading YAML file: {exc}")
            return None

# Function to execute the query
def run_query(engine,query):
    try:
        with engine.connect() as connection:
            result = connection.execute(text(query))
            return result.fetchall()
    except Exception as e:
        print(f"Error in the query -> {e}")

In [4]:
## Reading Configuration file

file_path = os.path.join(root_dir,'MYSQL_user_config.yaml')
config = read_yaml(file_path)

user = config.get('MYSQL_credentials')['user']
password = config.get('MYSQL_credentials')['password']
host = config.get('MYSQL_credentials')['host']
port = config.get('MYSQL_credentials')['port']

In [5]:
## Engine Configuration

db_url = f'mysql+pymysql://{user}:{password}@{host}:{port}'
db_name = "FETCH_DB_Amarthya"
engine = create_engine(f'{db_url}'+'/'+f'{db_name}')
run_query(engine, f'USE {db_name}')

Error in the query -> This result object does not return rows. It has been closed automatically.


### What are the top 5 brands by receipts scanned for most recent month?

Brands with the highest number of occurrences on all receipts: This measures the total number of times items from the brand appear across all receipts, counting every instance where an item from the brand is scanned.

Note that, as discussed in the Data Quality Section about brand and BrandCodes, since the best possible way to relate receipts and brands is by brandCode which is half empty, the imputing method of extracting the first word from description is used below. Additionally, brandCode is still treated as the primary identifier for naming.

The most recent month is 2021-01-03, which is dynamically calculated and not hard-coded. The query below ensures this is handled appropriately.

In [6]:
#### For most recent month:

query_1 = """
WITH COUNT_BRAND_RECEIPTS AS(
    SELECT RR.brandCode AS Brand,
    COUNT(RR.receipt_id) AS ReceiptCount,
    DENSE_RANK() OVER(ORDER BY COUNT(RR.receipt_id) DESC) AS BrandRank
    FROM rewards_receipts RR
    INNER JOIN receipts R
    ON RR.receipt_id = R.receipt_id
    WHERE DATE_FORMAT(R.scanned_date_time,'%Y%m') = (SELECT DATE_FORMAT(MAX(scanned_date_time),'%Y%m') FROM receipts) 
        AND RR.brandCode IS NOT NULL 
        AND RR.brandCode != 'ITEM'
    GROUP BY Brand
)

SELECT Brand, ReceiptCount, BrandRank
FROM COUNT_BRAND_RECEIPTS
WHERE BrandRank <= 5
ORDER BY BrandRank ASC;
"""

results=run_query(engine,query_1)

columns = ["brandCode", "Receipts Scanned", "Brand_Rank"]
table = PrettyTable()
table.field_names = columns
print('For the Most Recent Month:')
for row in results:
    table.add_row(row)

print(table)

For the Most Recent Month:
+-----------+------------------+------------+
| brandCode | Receipts Scanned | Brand_Rank |
+-----------+------------------+------------+
|  MUELLER  |        11        |     1      |
|  THINDUST |        11        |     1      |
+-----------+------------------+------------+


Since there are not atleast 5 distinct Brands, We could explore the top 5 brands for previous months.

#### For month = '2021-02'

In [7]:
## Hard coded month
set_month = "'2021-02-01'"
set_most_recent_month_query = f"SET @MOST_RECENT_MONTH = {set_month};"
run_query(engine,query = set_most_recent_month_query)

query_1 = """
WITH COUNT_BRAND_RECEIPTS AS(
    SELECT RR.brandCode AS Brand,
    COUNT(RR.receipt_id) AS ReceiptCount,
    DENSE_RANK() OVER(ORDER BY COUNT(RR.receipt_id) DESC) AS BrandRank
    FROM rewards_receipts RR
    INNER JOIN receipts R
    ON RR.receipt_id = R.receipt_id
    WHERE DATE_FORMAT(R.scanned_date_time,'%Y%m') = DATE_FORMAT(@MOST_RECENT_MONTH, '%Y%m')
      AND RR.brandCode IS NOT NULL 
      AND RR.brandCode != 'ITEM'
    GROUP BY Brand
)

SELECT Brand, ReceiptCount, BrandRank
FROM COUNT_BRAND_RECEIPTS
WHERE BrandRank <= 5
ORDER BY BrandRank ASC;
"""
results=run_query(engine,query_1)

columns = ["brandCode", "Receipts Scanned", "Brand_Rank"]
table = PrettyTable()
table.field_names = columns
print(f'For the Month: {set_month}')
for row in results:
    table.add_row(row)

print(table)

Error in the query -> This result object does not return rows. It has been closed automatically.
For the Month: '2021-02-01'
+-----------+------------------+------------+
| brandCode | Receipts Scanned | Brand_Rank |
+-----------+------------------+------------+
|  MUELLER  |        29        |     1      |
|  THINDUST |        29        |     1      |
|  FLIPBELT |        28        |     2      |
|   HEINZ   |        10        |     3      |
|   BRAND   |        3         |     4      |
|  DORITOS  |        3         |     4      |
|  DELETED  |        2         |     5      |
|    JIF    |        2         |     5      |
|   SUAVE   |        2         |     5      |
|  MISSION  |        2         |     5      |
|   CAPRI   |        2         |     5      |
|   SPIGEN  |        2         |     5      |
+-----------+------------------+------------+


From the above list, brands like 'DELETED','BRAND' do not make sense. This is due to the limitation of the imputation method discussed earlier. In some cases, the brand codes may not be accurate when extracted from the description.

#### For Month = '202101'

In [8]:
## Hard coded month
set_month = "'2021-01-01'"
set_most_recent_month_query = f"SET @MOST_RECENT_MONTH = {set_month};"
run_query(engine,query = set_most_recent_month_query)

query_1 = """
WITH COUNT_BRAND_RECEIPTS AS(
    SELECT RR.brandCode AS Brand,
    COUNT(RR.receipt_id) AS ReceiptCount,
    DENSE_RANK() OVER(ORDER BY COUNT(RR.receipt_id) DESC) AS BrandRank
    FROM rewards_receipts RR
    INNER JOIN receipts R
    ON RR.receipt_id = R.receipt_id
    WHERE DATE_FORMAT(R.scanned_date_time,'%Y%m') = DATE_FORMAT(@MOST_RECENT_MONTH, '%Y%m')
        AND RR.brandCode IS NOT NULL
        AND RR.brandCode != 'ITEM'
    GROUP BY Brand
)

SELECT Brand, ReceiptCount, BrandRank
FROM COUNT_BRAND_RECEIPTS
WHERE BrandRank <= 5
ORDER BY BrandRank ASC;
"""
results=run_query(engine,query_1)

columns = ["brandCode", "Receipts Scanned", "Brand_Rank"]
table = PrettyTable()
table.field_names = columns
print(f'For the Month: {set_month}')
for row in results:
    table.add_row(row)

print(table)

Error in the query -> This result object does not return rows. It has been closed automatically.
For the Month: '2021-01-01'
+----------------+------------------+------------+
|   brandCode    | Receipts Scanned | Brand_Rank |
+----------------+------------------+------------+
|     HY-VEE     |       296        |     1      |
| BEN AND JERRYS |       180        |     2      |
|       PC       |       138        |     3      |
|   KLARBRUNN    |       133        |     4      |
|     PEPSI      |       103        |     5      |
+----------------+------------------+------------+


The brand 'PC' might not be reasonable. This is due to the limitation of the imputation method discussed earlier. In some cases, the brand codes may not be accurate when extracted from the description.

### 2) How does the ranking of the top 5 brands by receipts scanned for the recent month compare to the ranking for the previous month

#### For the most Recent month

In [9]:
set_most_recent_month_query = """SET @MOST_RECENT_MONTH = (SELECT DATE_FORMAT(MAX(scanned_date_time), '%Y-%m-01') FROM receipts);"""
run_query(engine,query = set_most_recent_month_query)

query_2 = """
WITH CURRENT_RANK_TABLE AS (
    SELECT RR.brandCode AS Brand,
    SUM(IF(
    DATE_FORMAT(R.scanned_date_time,'%Y%m') = DATE_FORMAT(@MOST_RECENT_MONTH,'%Y%m'),1,0
    )) AS Current_Month_ReceiptsCount,
    SUM(IF(
    DATE_FORMAT(R.scanned_date_time, '%Y%m') = DATE_FORMAT(DATE_SUB(@MOST_RECENT_MONTH, INTERVAL 1 MONTH), '%Y%m'), 1, 0
    )) AS Previous_Month_ReceiptsCount,
    
    -- Apply DENSE_RANK() for Current Month Count
    DENSE_RANK() OVER 
    (ORDER BY SUM(IF(DATE_FORMAT(R.scanned_date_time,'%Y%m') = DATE_FORMAT(@MOST_RECENT_MONTH,'%Y%m'), 1, 0)) DESC)
    AS Current_Brand_Rank,

    -- Apply DENSE_RANK() for Previous Month Count
    DENSE_RANK() OVER 
    (ORDER BY SUM(IF(DATE_FORMAT(R.scanned_date_time, '%Y%m') = DATE_FORMAT(DATE_SUB(@MOST_RECENT_MONTH, INTERVAL 1 MONTH), '%Y%m'), 1, 0)) DESC)
    AS Previous_Brand_Rank

    FROM rewards_receipts RR
    INNER JOIN receipts R
    ON RR.receipt_id = R.receipt_id
    WHERE RR.brandCode IS NOT NULL AND RR.brandCode != 'ITEM'
    GROUP BY Brand
)
SELECT Brand, 
Current_Month_ReceiptsCount, 
Previous_Month_ReceiptsCount, 
Current_Brand_Rank, 
Previous_Brand_Rank, 
CAST(Previous_Brand_Rank AS SIGNED) - CAST(Current_Brand_Rank AS SIGNED) Rank_Difference
FROM CURRENT_RANK_TABLE
WHERE Current_Brand_Rank <= 5 AND Current_Month_ReceiptsCount!= 0
ORDER BY Current_Brand_Rank ASC;
"""
results=run_query(engine,query_2)

columns = ["Brand", "Current_Month_ReceiptsCount","Previous_Month_ReceiptsCount","Current_Month_Rank", "Previous_Month_Rank","rank_Difference"]
table = PrettyTable()
table.field_names = columns
print('For the most recent Month:')
for row in results:
    table.add_row(row)

print(table)

Error in the query -> This result object does not return rows. It has been closed automatically.
For the most recent Month:
+----------+-----------------------------+------------------------------+--------------------+---------------------+-----------------+
|  Brand   | Current_Month_ReceiptsCount | Previous_Month_ReceiptsCount | Current_Month_Rank | Previous_Month_Rank | rank_Difference |
+----------+-----------------------------+------------------------------+--------------------+---------------------+-----------------+
| MUELLER  |              11             |              29              |         1          |          1          |        0        |
| THINDUST |              11             |              29              |         1          |          1          |        0        |
+----------+-----------------------------+------------------------------+--------------------+---------------------+-----------------+


#### For Month: 2021-02

In [10]:
set_month = "'2021-02-01'"
set_most_recent_month_query = f"SET @MOST_RECENT_MONTH = {set_month};"
run_query(engine,query = set_most_recent_month_query)

query_2 = """
WITH CURRENT_RANK_TABLE AS (
    SELECT RR.brandCode AS Brand,
    SUM(IF(
    DATE_FORMAT(R.scanned_date_time,'%Y%m') = DATE_FORMAT(@MOST_RECENT_MONTH,'%Y%m'),1,0
    )) AS Current_Month_ReceiptsCount,
    SUM(IF(
    DATE_FORMAT(R.scanned_date_time, '%Y%m') = DATE_FORMAT(DATE_SUB(@MOST_RECENT_MONTH, INTERVAL 1 MONTH), '%Y%m'), 1, 0
    )) AS Previous_Month_ReceiptsCount,
    
    -- Apply DENSE_RANK() for Current Month Count
    DENSE_RANK() OVER 
    (ORDER BY SUM(IF(DATE_FORMAT(R.scanned_date_time,'%Y%m') = DATE_FORMAT(@MOST_RECENT_MONTH,'%Y%m'), 1, 0)) DESC)
    AS Current_Brand_Rank,

    -- Apply DENSE_RANK() for Previous Month Count
    DENSE_RANK() OVER 
    (ORDER BY SUM(IF(DATE_FORMAT(R.scanned_date_time, '%Y%m') = DATE_FORMAT(DATE_SUB(@MOST_RECENT_MONTH, INTERVAL 1 MONTH), '%Y%m'), 1, 0)) DESC)
    AS Previous_Brand_Rank

    FROM rewards_receipts RR
    INNER JOIN receipts R
    ON RR.receipt_id = R.receipt_id
    WHERE RR.brandCode IS NOT NULL AND RR.brandCode != 'ITEM'
    GROUP BY Brand
)
SELECT Brand, 
Current_Month_ReceiptsCount, 
Previous_Month_ReceiptsCount, 
Current_Brand_Rank, 
Previous_Brand_Rank, 
CAST(Previous_Brand_Rank AS SIGNED) - CAST(Current_Brand_Rank AS SIGNED) Rank_Difference
FROM CURRENT_RANK_TABLE
WHERE Current_Brand_Rank <= 5 AND Current_Month_ReceiptsCount!= 0
ORDER BY Current_Brand_Rank ASC;
"""
results=run_query(engine,query_2)

columns = ["Brand", "Current_Month_ReceiptsCount","Previous_Month_ReceiptsCount","Current_Month_Rank", "Previous_Month_Rank","rank_Difference"]
table = PrettyTable()
table.field_names = columns
print(f'For the most recent Month as {set_month}:')
for row in results:
    table.add_row(row)

print(table)

Error in the query -> This result object does not return rows. It has been closed automatically.
For the most recent Month as '2021-02-01':
+----------+-----------------------------+------------------------------+--------------------+---------------------+-----------------+
|  Brand   | Current_Month_ReceiptsCount | Previous_Month_ReceiptsCount | Current_Month_Rank | Previous_Month_Rank | rank_Difference |
+----------+-----------------------------+------------------------------+--------------------+---------------------+-----------------+
| MUELLER  |              29             |              4               |         1          |          51         |        50       |
| THINDUST |              29             |              4               |         1          |          51         |        50       |
| FLIPBELT |              28             |              22              |         2          |          33         |        31       |
|  HEINZ   |              10             |        

#### For Month 2021-01:

In [11]:
set_month = "'2021-01-01'"
set_most_recent_month_query = f"SET @MOST_RECENT_MONTH = {set_month};"
run_query(engine,query = set_most_recent_month_query)

query_2 = """
WITH CURRENT_RANK_TABLE AS (
    SELECT RR.brandCode AS Brand,
    SUM(IF(
    DATE_FORMAT(R.scanned_date_time,'%Y%m') = DATE_FORMAT(@MOST_RECENT_MONTH,'%Y%m'),1,0
    )) AS Current_Month_ReceiptsCount,
    SUM(IF(
    DATE_FORMAT(R.scanned_date_time, '%Y%m') = DATE_FORMAT(DATE_SUB(@MOST_RECENT_MONTH, INTERVAL 1 MONTH), '%Y%m'), 1, 0
    )) AS Previous_Month_ReceiptsCount,
    
    -- Apply DENSE_RANK() for Current Month Count
    DENSE_RANK() OVER 
    (ORDER BY SUM(IF(DATE_FORMAT(R.scanned_date_time,'%Y%m') = DATE_FORMAT(@MOST_RECENT_MONTH,'%Y%m'), 1, 0)) DESC)
    AS Current_Brand_Rank,

    -- Apply DENSE_RANK() for Previous Month Count
    DENSE_RANK() OVER 
    (ORDER BY SUM(IF(DATE_FORMAT(R.scanned_date_time, '%Y%m') = DATE_FORMAT(DATE_SUB(@MOST_RECENT_MONTH, INTERVAL 1 MONTH), '%Y%m'), 1, 0)) DESC)
    AS Previous_Brand_Rank

    FROM rewards_receipts RR
    INNER JOIN receipts R
    ON RR.receipt_id = R.receipt_id
    WHERE RR.brandCode IS NOT NULL AND RR.brandCode != 'ITEM'
    GROUP BY Brand
)
SELECT Brand, 
Current_Month_ReceiptsCount, 
Previous_Month_ReceiptsCount, 
Current_Brand_Rank, 
Previous_Brand_Rank, 
CAST(Previous_Brand_Rank AS SIGNED) - CAST(Current_Brand_Rank AS SIGNED) Rank_Difference
FROM CURRENT_RANK_TABLE
WHERE Current_Brand_Rank <= 5 AND Current_Month_ReceiptsCount!= 0
ORDER BY Current_Brand_Rank ASC;
"""
results=run_query(engine,query_2)

columns = ["Brand", "Current_Month_ReceiptsCount","Previous_Month_ReceiptsCount","Current_Month_Rank", "Previous_Month_Rank","rank_Difference"]
table = PrettyTable()
table.field_names = columns
print(f'For the most recent Month as {set_month}:')
for row in results:
    table.add_row(row)

print(table)

Error in the query -> This result object does not return rows. It has been closed automatically.
For the most recent Month as '2021-01-01':
+----------------+-----------------------------+------------------------------+--------------------+---------------------+-----------------+
|     Brand      | Current_Month_ReceiptsCount | Previous_Month_ReceiptsCount | Current_Month_Rank | Previous_Month_Rank | rank_Difference |
+----------------+-----------------------------+------------------------------+--------------------+---------------------+-----------------+
|     HY-VEE     |             296             |              0               |         1          |          1          |        0        |
| BEN AND JERRYS |             180             |              0               |         2          |          1          |        -1       |
|       PC       |             138             |              0               |         3          |          1          |        -2       |
|   KLARBRUNN 